In [1]:
#IMPORT STATEMENTS
import pandas as pd
import json
import nltk
from nltk.tokenize import sent_tokenize
import os
import transformers
import torch
from transformers import pipeline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
import json

def clean_keys(obj):
    """
    Recursively clean the keys in a dict or list by removing
    extra starting and ending double quotes.
    """
    if isinstance(obj, dict):
        new_dict = {}
        for key, value in obj.items():
            # If key starts and ends with an extra quote, remove them.
            new_key = key
            if new_key.startswith('"') and new_key.endswith('"'):
                new_key = new_key[1:-1]
            new_dict[new_key] = clean_keys(value)
        return new_dict
    elif isinstance(obj, list):
        return [clean_keys(item) for item in obj]
    else:
        return obj

# If your JSON is stored in a file, you can load it like this:
with open('C:\\Users\\Harshita\\Downloads\\case study\\metadata_schema.json', encoding='utf-8') as f:
    data = json.load(f)

cleaned_data = clean_keys(data)

# Print the cleaned JSON with readable Unicode characters.
print(json.dumps(cleaned_data, ensure_ascii=False, indent=2))
open('cleaned_data.json', 'w', encoding='utf-8').write(json.dumps(cleaned_data, ensure_ascii=False, indent=2))



[
  {
    "Standort": "Gelsenkirchen-Buer | Hemer im Sauerland | Ilmenau | Mülheim an der Möhne",
    "Archiv ID": null,
    "PROBANDNR": null,
    "DOK_ART": null,
    "ARCHIVORT": null,
    "PROVENIENZ": null,
    "SPERRUNG": null,
    "ENTSTZEIT": null,
    "Zeitumfang 1": null,
    "NAME": null,
    "VORNAME": null,
    "ORT": "Gelsenkirchen-Buer | Gelsenkirchen | Hemer im Sauerland | Belecke | Dortmund | Ilmenau | Allagen | Mülheim an der Möhne | Wimpfen am Neckar",
    "Feld1": null,
    "PSEUDONYM": null,
    "GESCHLECHT": "weiblich",
    "JAHRGANG": 1925,
    "IPV": null,
    "DATENBOGEN": null,
    "KURZBESCHR": null,
    "TITEL": null,
    "STRASSE": "Vom-Stein-Straße",
    "PLZ": null,
    "TELEFON": null,
    "GRUPPE": "Arbeitsdienst | NSV",
    "BERUF": "Arbeitsführer",
    "HEUT_FAMST": "Witwe | verheiratet",
    "INTERVIEWE": null,
    "TIPPER": null,
    "Segmentierung": null,
    "DATUM1": "November 1942",
    "DATUM2": null,
    "DATUM3": null,
    "DAUER": null,
    

3643

# RUN

In [45]:
with open('cleaned_data.json', encoding='utf-8') as f:
    data = json.load(f)

cleaned_data = clean_keys(data)
cleaned_data

[{'Standort': 'Gelsenkirchen-Buer | Hemer im Sauerland | Ilmenau | Mülheim an der Möhne',
  'Archiv ID': None,
  'PROBANDNR': None,
  'DOK_ART': None,
  'ARCHIVORT': None,
  'PROVENIENZ': None,
  'SPERRUNG': None,
  'ENTSTZEIT': None,
  'Zeitumfang 1': None,
  'NAME': None,
  'VORNAME': None,
  'ORT': 'Gelsenkirchen-Buer | Gelsenkirchen | Hemer im Sauerland | Belecke | Dortmund | Ilmenau | Allagen | Mülheim an der Möhne | Wimpfen am Neckar',
  'Feld1': None,
  'PSEUDONYM': None,
  'GESCHLECHT': 'weiblich',
  'JAHRGANG': 1925,
  'IPV': None,
  'DATENBOGEN': None,
  'KURZBESCHR': None,
  'TITEL': None,
  'STRASSE': 'Vom-Stein-Straße',
  'PLZ': None,
  'TELEFON': None,
  'GRUPPE': 'Arbeitsdienst | NSV',
  'BERUF': 'Arbeitsführer',
  'HEUT_FAMST': 'Witwe | verheiratet',
  'INTERVIEWE': None,
  'TIPPER': None,
  'Segmentierung': None,
  'DATUM1': 'November 1942',
  'DATUM2': None,
  'DATUM3': None,
  'DAUER': None,
  'online': None,
  'AUSDRUCKSART': None,
  'UNKAUSDRUC': None,
  'KORRAUSDR

In [34]:
def process_metadata(metadata_pipeline, json_data, transcript, context_window):
    """
    Processes a cleaned metadata JSON object by selecting the best candidate response 
    for each field using LLaMA-based ranking.

    Args:
        metadata_pipeline: The LLaMA model pipeline for text generation.
        json_data (dict or list): The cleaned metadata JSON data.
        transcript (str): The full transcript text.
        context_window (int): Maximum number of characters from the transcript to include.

    Returns:
        dict or list: The metadata JSON with each field replaced by the best response.
    """
    def process_item(item):
        for key, value in item.items():
            # Process only string values
            if isinstance(value, str):
                # If the field is empty, set it to blank.
                if not value.strip():
                    item[key] = ""
                else:
                    # Split candidate responses using the pipe separator.
                    candidates = [c.strip() for c in value.split("|") if c.strip()]
                    # If there's only one candidate, keep it.
                    if len(candidates) == 1:
                        item[key] = candidates[0]
                    else:
                        # Build a list of candidate responses.
                        candidate_lines = "\n".join([f"- {c}" for c in candidates])
                        # Use the last 'context_window' characters of the transcript.
                        transcript_context = transcript[-context_window:] if len(transcript) > context_window else transcript

                        prompt = f"""
You are an AI that must provide exactly ONE short answer for the metadata field '{key}'.
Context:
{transcript_context}

Candidate Responses:
{candidate_lines}

Rules:
- If one candidate is clearly correct, return it.
- If multiple are plausible, return the one best supported by the context If not, return all the candidates.
- If none of the responses is contextually appropriate, return an empty string.

Output must be a SINGLE line with NO extra explanations or repetition.
"""

                        # Query the LLM using the provided pipeline.
                        result = metadata_pipeline(prompt, max_new_tokens=1800, return_full_text=False, temperature=0.2)
                        response = result[0]["generated_text"].strip()
                        print(f"    -> LLaMA response: {response}")
                        item[key] = response
            else:
                # For non-string values, set to an empty string.
                item[key] = ""
        return item

    # Process a list of records or a single record.
    if isinstance(json_data, list):
        return [process_item(item) for item in json_data]
    elif isinstance(json_data, dict):
        return process_item(json_data)
    else:
        raise ValueError("Unsupported JSON data structure. Expected a dict or list of dicts.")


In [42]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

model_path = "models/meta-llama/Llama-3.3-70B-Instruct"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map="auto", 
    torch_dtype=torch.bfloat16, 
    quantization_config=quantization_config,
    trust_remote_code=True  # Add this for some custom models
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

metadata_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [40]:
file_path = 'Transcripts/adg0001_er_2024_10_31.csv'
input_data = pd.read_csv(file_path, sep=None, engine='python')
transcript = " ".join(input_data["Transkript"].dropna().astype(str))

final_cleaned_json = process_metadata(metadata_pipeline, cleaned_data, transcript, context_window=1800)


In [33]:
print(json.dumps(final_cleaned_json, ensure_ascii=False, indent=2))

[
  {
    "Standort": "- Gelsenkirchen-Buer \nis not contextually relevant. \nThe most contextually relevant response is \n- Mülheim an der Möhne. \nTherefore, the response is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer is \nMülheim an der Möhne. \nThe final answer